# Project 2: Supervised Learning
### Building a Student Intervention System

## 1. Classification vs Regression

Your goal is to identify students who might need early intervention - which type of supervised machine learning problem is this, classification or regression? Why?


|  Answer |
|------------|
|This should be a classification problem as we group students based on whether they need early intervention or not. |

## 2. Exploring the Data

Let's go ahead and read in the student dataset first.

_To execute a code cell, click inside it and press **Shift+Enter**._

In [1]:
# Import libraries
import numpy as np
import pandas as pd

In [2]:
# Read student data
student_data = pd.read_csv("student-data.csv")
print "Student data read successfully!"
# Note: The last column 'passed' is the target/label, all other are feature columns

Student data read successfully!


Now, can you find out the following facts about the dataset?
- Total number of students
- Number of students who passed
- Number of students who failed
- Graduation rate of the class (%)
- Number of features

_Use the code block below to compute these values. Instructions/steps are marked using **TODO**s._

In [3]:
# TODO: Compute desired values - replace each '?' with an appropriate expression/function call
n_students = len(student_data)
n_features = len(student_data.columns)
n_passed = len(student_data.loc[student_data['passed'] == 'yes'])
n_failed = len(student_data.loc[student_data['passed'] == 'no'])
grad_rate = float(n_passed)/float(n_students)*100
print "Total number of students: {}".format(n_students)
print "Number of students who passed: {}".format(n_passed)
print "Number of students who failed: {}".format(n_failed)
print "Number of features: {}".format(n_features)
print "Graduation rate of the class: {:.2f}%".format(grad_rate)

Total number of students: 395
Number of students who passed: 265
Number of students who failed: 130
Number of features: 31
Graduation rate of the class: 67.09%


## 3. Preparing the Data
In this section, we will prepare the data for modeling, training and testing.

### Identify feature and target columns
It is often the case that the data you obtain contains non-numeric features. This can be a problem, as most machine learning algorithms expect numeric data to perform computations with.

Let's first separate our data into feature and target columns, and see if any features are non-numeric.<br/>
**Note**: For this dataset, the last column (`'passed'`) is the target or label we are trying to predict.

In [4]:
# Extract feature (X) and target (y) columns
feature_cols = list(student_data.columns[:-1])  # all columns but last are features
target_col = student_data.columns[-1]  # last column is the target/label
print "Feature column(s):-\n{}".format(feature_cols)
print "Target column: {}".format(target_col)

X_all = student_data[feature_cols]  # feature values for all students
y_all = student_data[target_col]  # corresponding targets/labels
print "\nFeature values:-"
print X_all.head()  # print the first 5 rows

Feature column(s):-
['school', 'sex', 'age', 'address', 'famsize', 'Pstatus', 'Medu', 'Fedu', 'Mjob', 'Fjob', 'reason', 'guardian', 'traveltime', 'studytime', 'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic', 'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences']
Target column: passed

Feature values:-
  school sex  age address famsize Pstatus  Medu  Fedu     Mjob      Fjob  \
0     GP   F   18       U     GT3       A     4     4  at_home   teacher   
1     GP   F   17       U     GT3       T     1     1  at_home     other   
2     GP   F   15       U     LE3       T     1     1  at_home     other   
3     GP   F   15       U     GT3       T     4     2   health  services   
4     GP   F   16       U     GT3       T     3     3    other     other   

    ...    higher internet  romantic  famrel  freetime goout Dalc Walc health  \
0   ...       yes       no        no       4         3     4    1    1      3   
1   ...    

### Preprocess feature columns

As you can see, there are several non-numeric columns that need to be converted! Many of them are simply `yes`/`no`, e.g. `internet`. These can be reasonably converted into `1`/`0` (binary) values.

Other columns, like `Mjob` and `Fjob`, have more than two values, and are known as _categorical variables_. The recommended way to handle such a column is to create as many columns as possible values (e.g. `Fjob_teacher`, `Fjob_other`, `Fjob_services`, etc.), and assign a `1` to one of them and `0` to all others.

These generated columns are sometimes called _dummy variables_, and we will use the [`pandas.get_dummies()`](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.get_dummies.html?highlight=get_dummies#pandas.get_dummies) function to perform this transformation.

In [5]:
# Preprocess feature columns
def preprocess_features(X):
    outX = pd.DataFrame(index=X.index)  # output dataframe, initially empty

    # Check each column
    for col, col_data in X.iteritems():
        # If data type is non-numeric, try to replace all yes/no values with 1/0
        if col_data.dtype == object:
            col_data = col_data.replace(['yes', 'no'], [1, 0])
        # Note: This should change the data type for yes/no columns to int

        # If still non-numeric, convert to one or more dummy variables
        if col_data.dtype == object:
            col_data = pd.get_dummies(col_data, prefix=col)  # e.g. 'school' => 'school_GP', 'school_MS'

        outX = outX.join(col_data)  # collect column(s) in output dataframe

    return outX

X_all = preprocess_features(X_all)
print "Processed feature columns ({}):-\n{}".format(len(X_all.columns), list(X_all.columns))

Processed feature columns (48):-
['school_GP', 'school_MS', 'sex_F', 'sex_M', 'age', 'address_R', 'address_U', 'famsize_GT3', 'famsize_LE3', 'Pstatus_A', 'Pstatus_T', 'Medu', 'Fedu', 'Mjob_at_home', 'Mjob_health', 'Mjob_other', 'Mjob_services', 'Mjob_teacher', 'Fjob_at_home', 'Fjob_health', 'Fjob_other', 'Fjob_services', 'Fjob_teacher', 'reason_course', 'reason_home', 'reason_other', 'reason_reputation', 'guardian_father', 'guardian_mother', 'guardian_other', 'traveltime', 'studytime', 'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic', 'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences']


In [6]:
# Preprocess label data
#y_all = y_all.replace(['yes','no'],[1,0])

### Split data into training and test sets

So far, we have converted all _categorical_ features into numeric values. In this next step, we split the data (both features and corresponding labels) into training and test sets.

In [7]:
# First, decide how many training vs test samples you want
num_all = student_data.shape[0]  # same as len(student_data)
num_train = 300  # about 75% of the data
num_test = num_all - num_train

# TODO: Then, select features (X) and corresponding labels (y) for the training and test sets
# Note: Shuffle the data or randomly select samples to avoid any bias due to ordering in the dataset
np.random.seed(0) 
indices = np.random.permutation(num_all)
training_idx, test_idx = indices[:num_train], indices[num_train:]

X_train = X_all.loc[training_idx]
y_train = y_all.loc[training_idx]
X_test = X_all.loc[test_idx]
y_test = y_all.loc[test_idx]
print "Training set: {} samples".format(X_train.shape[0])
print "Test set: {} samples".format(X_test.shape[0])
# Note: If you need a validation set, extract it from within training data

Training set: 300 samples
Test set: 95 samples


## 4. Training and Evaluating Models
Choose 3 supervised learning models that are available in scikit-learn, and appropriate for this problem. For each model:

- What are the general applications of this model? What are its strengths and weaknesses?
- Given what you know about the data so far, why did you choose this model to apply?
- Fit this model to the training data, try to predict labels (for both training and test sets), and measure the F<sub>1</sub> score. Repeat this process with different training set sizes (100, 200, 300), keeping test set constant.

Produce a table showing training time, prediction time, F<sub>1</sub> score on training set and F<sub>1</sub> score on test set, for each training set size.

Note: You need to produce 3 such tables - one for each model.

### 4-1 SVM 

| What are the general applications of this model? What are its strengths and weaknesses? |
| -----|
| General applications: text categorization, image classification, hand writing recognition and etc.|
| Strength: 1) Training is easy because no local optimum solution, 2) works relatively well with small sample sizes, 3) kernel-based model is very powerful and flexible|
| Weakness: 1) Sensitive to noise, 2) need a good kerner function, 3) no direct multi-class classfication with SVM, 4) computationally expensive|

| Given what you know about the data so far, why did you choose this model to apply? |
| -----|
| I found a reference that SVM works well with small sample sizes. The size of sample to train the model for this project is only 300, therefore I thought this may be a good model to start with.|

| Fit this model to the training data, try to predict labels (for both training and test sets), and measure the F<sub>1</sub> score. Repeat this process with different training set sizes (100, 200, 300), keeping test set constant.|
| -----|
| Please see below.|

In [8]:
# Train a model
import time

def train_classifier(clf, X_train, y_train):
    print "Training {}...".format(clf.__class__.__name__)
    start = time.time()
    clf.fit(X_train, y_train)
    end = time.time()
    print "Done!\nTraining time (secs): {:.3f}".format(end - start)
    print "------------------------------------------"
# TODO: Choose a model, import it and instantiate an object
from sklearn import svm
clf = svm.SVC(kernel = 'rbf', C=10.0)

# Fit model to training data
train_classifier(clf, X_train, y_train)  # note: using entire training set here
print clf  # you can inspect the learned model by printing it

Training SVC...
Done!
Training time (secs): 0.014
------------------------------------------
SVC(C=10.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)


In [9]:
# Predict on training set and compute F1 score
from sklearn.metrics import f1_score

def predict_labels(clf, features, target):
    print "Predicting labels using {}...".format(clf.__class__.__name__)
    start = time.time()
    y_pred = clf.predict(features)
    end = time.time()
    print "Done!\nPrediction time (secs): {:.3f}".format(end - start)
    return f1_score(target.values, y_pred, pos_label='yes')

train_f1_score = predict_labels(clf, X_train, y_train)
print "F1 score for training set: {}".format(train_f1_score)

Predicting labels using SVC...
Done!
Prediction time (secs): 0.007
F1 score for training set: 0.984848484848


In [10]:
# Predict on test data
print "F1 score for test set: {}".format(predict_labels(clf, X_test, y_test))

Predicting labels using SVC...
Done!
Prediction time (secs): 0.003
F1 score for test set: 0.780141843972


In [11]:
# Train and predict using different training set sizes
def train_predict(clf, X_train, y_train, X_test, y_test):
    print "------------------------------------------"
    print "Training set size: {}".format(len(X_train))
    train_classifier(clf, X_train, y_train)
    print "F1 score for training set: {}".format(predict_labels(clf, X_train, y_train))
    print "F1 score for test set: {}".format(predict_labels(clf, X_test, y_test))

# TODO: Run the helper function above for desired subsets of training data
# Note: Keep the test set constant

#### Differentiate training data set size: [100, 200, 300]

* data set size = 100

In [12]:
train_predict(clf,X_train[:100],y_train[:100],X_test,y_test)

------------------------------------------
Training set size: 100
Training SVC...
Done!
Training time (secs): 0.003
------------------------------------------
Predicting labels using SVC...
Done!
Prediction time (secs): 0.004
F1 score for training set: 1.0
Predicting labels using SVC...
Done!
Prediction time (secs): 0.002
F1 score for test set: 0.774647887324


* data set size = 200

In [13]:
train_predict(clf,X_train[:200],y_train[:200],X_test,y_test)

------------------------------------------
Training set size: 200
Training SVC...
Done!
Training time (secs): 0.007
------------------------------------------
Predicting labels using SVC...
Done!
Prediction time (secs): 0.004
F1 score for training set: 0.992063492063
Predicting labels using SVC...
Done!
Prediction time (secs): 0.002
F1 score for test set: 0.764705882353


* data set size = 300

In [14]:
train_predict(clf,X_train[:300],y_train[:300],X_test,y_test)

------------------------------------------
Training set size: 300
Training SVC...
Done!
Training time (secs): 0.014
------------------------------------------
Predicting labels using SVC...
Done!
Prediction time (secs): 0.007
F1 score for training set: 0.984848484848
Predicting labels using SVC...
Done!
Prediction time (secs): 0.003
F1 score for test set: 0.780141843972


| SVM: Training set size| 100 | 200| 300|
| ----- |
| Training time (secs)|0.001 |0.003|0.006|
|Prediction time (secs)|0.001 |0.001|0.002|
|F1 score for training set| 1.0|0.992|0.985|
|F1 score for test set| 0.775|0.765|0.780|

### 4-2 Decision Tree

| What are the general applications of this model? What are its strengths and weaknesses? |
| -----|
| General applications: classification examples such as fraud detection in emails, credit card usage or customer segregation for a website or an eCommerce company. It can be used where the end results can be multiple classification groups and not binary.  |
| Strength: 1) Easy to interpret (white box model), 2) Works on both numerical and categorical data, 3) Tree does automatic feature selection 4) Requires little data preparation like normaliazation before processing. |
| Weakness: 1) Prone to overfitting, 2) Calculation gets slower as dataset become larger, 3) Inadequate in predicting continuous values, 4) Greedy classifier (may not find the best tree all the time) |

| Given what you know about the data so far, why did you choose this model to apply? |
| -----|
|Decision Tree is chosen because it is one of the easiest model to interpret the results and because it is considered to be a faster classifier.|

| Fit this model to the training data, try to predict labels (for both training and test sets), and measure the F<sub>1</sub> score. Repeat this process with different training set sizes (100, 200, 300), keeping test set constant.|
| -----|
| Please see below.|

In [15]:
# Train a model
#import time

#def train_classifier(clf, X_train, y_train):
#    print "Training {}...".format(clf.__class__.__name__)
#    start = time.time()
#    clf.fit(X_train, y_train)
#    end = time.time()
#    print "Done!\nTraining time (secs): {:.3f}".format(end - start)
#    print "------------------------------------------"
## TODO: Choose a model, import it and instantiate an object



from sklearn import tree
clf = tree.DecisionTreeClassifier()

# Fit model to training data
train_classifier(clf, X_train, y_train)  # note: using entire training set here
#print clf  # you can inspect the learned model by printing it

Training DecisionTreeClassifier...
Done!
Training time (secs): 0.005
------------------------------------------


In [16]:
# Predict on training set and compute F1 score
#from sklearn.metrics import f1_score

#def predict_labels(clf, features, target):
#    print "Predicting labels using {}...".format(clf.__class__.__name__)
#    start = time.time()
#    y_pred = clf.predict(features)
#    end = time.time()
#    print "Done!\nPrediction time (secs): {:.4f}".format(end - start)
#    return f1_score(target.values, y_pred, pos_label='yes')
#

train_f1_score = predict_labels(clf, X_train, y_train)
print "F1 score for training set: {}".format(train_f1_score)

Predicting labels using DecisionTreeClassifier...
Done!
Prediction time (secs): 0.001
F1 score for training set: 1.0


In [17]:
# Predict on test data
print "F1 score for test set: {}".format(predict_labels(clf, X_test, y_test))

Predicting labels using DecisionTreeClassifier...
Done!
Prediction time (secs): 0.001
F1 score for test set: 0.742424242424


In [18]:
# Train and predict using different training set sizes
#def train_predict(clf, X_train, y_train, X_test, y_test):
#    print "------------------------------------------"
#    print "Training set size: {}".format(len(X_train))
#    train_classifier(clf, X_train, y_train)
#    print "F1 score for training set: {}".format(predict_labels(clf, X_train, y_train))
#    print "F1 score for test set: {}".format(predict_labels(clf, X_test, y_test))

# TODO: Run the helper function above for desired subsets of training data
# Note: Keep the test set constant

#### Differentiate training data set size: [100, 200, 300]

* data set size = 100

In [19]:
train_predict(clf,X_train[:100],y_train[:100],X_test,y_test)

------------------------------------------
Training set size: 100
Training DecisionTreeClassifier...
Done!
Training time (secs): 0.001
------------------------------------------
Predicting labels using DecisionTreeClassifier...
Done!
Prediction time (secs): 0.000
F1 score for training set: 1.0
Predicting labels using DecisionTreeClassifier...
Done!
Prediction time (secs): 0.000
F1 score for test set: 0.692307692308


* data set size = 200

In [20]:
train_predict(clf,X_train[:200],y_train[:200],X_test,y_test)

------------------------------------------
Training set size: 200
Training DecisionTreeClassifier...
Done!
Training time (secs): 0.002
------------------------------------------
Predicting labels using DecisionTreeClassifier...
Done!
Prediction time (secs): 0.000
F1 score for training set: 1.0
Predicting labels using DecisionTreeClassifier...
Done!
Prediction time (secs): 0.000
F1 score for test set: 0.778625954198


* data set size = 300

In [21]:
train_predict(clf,X_train[:300],y_train[:300],X_test,y_test)

------------------------------------------
Training set size: 300
Training DecisionTreeClassifier...
Done!
Training time (secs): 0.003
------------------------------------------
Predicting labels using DecisionTreeClassifier...
Done!
Prediction time (secs): 0.001
F1 score for training set: 1.0
Predicting labels using DecisionTreeClassifier...
Done!
Prediction time (secs): 0.000
F1 score for test set: 0.746268656716



|  Decision Tree: Training set size| 100 | 200| 300|
| ----- |
| Training time (secs)|0.000 |0.001|0.000|
|Prediction time (secs)| 0.000|0.000|0.000|
|F1 score for training set|1.0 |1.0|1.0|
|F1 score for test set|0.75 |0.779|0.746|

### 4-3 Adaboost

| What are the general applications of this model? What are its strengths and weaknesses? |
| -----|
| General applications: Biology, computer vision, speech processing |
| Strength: 1) Can be used with many different classifiers, 2) Not prone to overfitting, 3) Improves classification accuracy|
| Weakness: 1) Can be sensitive to noisy data and outliers, 2) May take longer than two above as this approach uses iterations until it reaches the optimal solution  |

| Given what you know about the data so far, why did you choose this model to apply? |
| -----|
|One of the shortcomings of the Decision Tree classifier is the model is prone to overfitting. I wanted to use Adaboost on DT classifier to see if how the ensemble method will improve the overfitting (f1 score).|

| Fit this model to the training data, try to predict labels (for both training and test sets), and measure the F<sub>1</sub> score. Repeat this process with different training set sizes (100, 200, 300), keeping test set constant.|
| -----|
| Please see below.|

In [22]:
# Train a model
#import time

#def train_classifier(clf, X_train, y_train):
#    print "Training {}...".format(clf.__class__.__name__)
#    start = time.time()
#    clf.fit(X_train, y_train)
#    end = time.time()
#    print "Done!\nTraining time (secs): {:.3f}".format(end - start)
#    print "------------------------------------------"
## TODO: Choose a model, import it and instantiate an object

from sklearn.ensemble import AdaBoostClassifier
clf = AdaBoostClassifier(n_estimators=200)

# Fit model to training data
train_classifier(clf, X_train, y_train)  # note: using entire training set here
#print clf  # you can inspect the learned model by printing it

Training AdaBoostClassifier...
Done!
Training time (secs): 0.462
------------------------------------------


In [23]:
# Predict on training set and compute F1 score
#from sklearn.metrics import f1_score

#def predict_labels(clf, features, target):
#    print "Predicting labels using {}...".format(clf.__class__.__name__)
#    start = time.time()
#    y_pred = clf.predict(features)
#    end = time.time()
#    print "Done!\nPrediction time (secs): {:.3f}".format(end - start)
#    
#    return f1_score(target.values, y_pred, pos_label='yes')

train_f1_score = predict_labels(clf, X_train, y_train)
print "F1 score for training set: {}".format(train_f1_score)

Predicting labels using AdaBoostClassifier...
Done!
Prediction time (secs): 0.035
F1 score for training set: 0.855721393035


In [24]:
# Predict on test data
print "F1 score for test set: {}".format(predict_labels(clf, X_test, y_test))

Predicting labels using AdaBoostClassifier...
Done!
Prediction time (secs): 0.030
F1 score for test set: 0.711111111111


In [25]:
# Train and predict using different training set sizes
#def train_predict(clf, X_train, y_train, X_test, y_test):
#    print "------------------------------------------"
#    print "Training set size: {}".format(len(X_train))
#    train_classifier(clf, X_train, y_train)
#    print "F1 score for training set: {}".format(predict_labels(clf, X_train, y_train))
#    print "F1 score for test set: {}".format(predict_labels(clf, X_test, y_test))

# TODO: Run the helper function above for desired subsets of training data
# Note: Keep the test set constant

#### Differentiate training data set size: [100, 200, 300]

* data set size = 100

In [26]:
train_predict(clf,X_train[:100],y_train[:100],X_test,y_test)

------------------------------------------
Training set size: 100
Training AdaBoostClassifier...
Done!
Training time (secs): 0.375
------------------------------------------
Predicting labels using AdaBoostClassifier...
Done!
Prediction time (secs): 0.017
F1 score for training set: 1.0
Predicting labels using AdaBoostClassifier...
Done!
Prediction time (secs): 0.017
F1 score for test set: 0.715328467153


* data set size = 200

In [27]:
train_predict(clf,X_train[:200],y_train[:200],X_test,y_test)

------------------------------------------
Training set size: 200
Training AdaBoostClassifier...
Done!
Training time (secs): 0.460
------------------------------------------
Predicting labels using AdaBoostClassifier...
Done!
Prediction time (secs): 0.046
F1 score for training set: 0.889763779528
Predicting labels using AdaBoostClassifier...
Done!
Prediction time (secs): 0.054
F1 score for test set: 0.727272727273


* data set size = 300

In [28]:
train_predict(clf,X_train[:300],y_train[:300],X_test,y_test)

------------------------------------------
Training set size: 300
Training AdaBoostClassifier...
Done!
Training time (secs): 0.536
------------------------------------------
Predicting labels using AdaBoostClassifier...
Done!
Prediction time (secs): 0.023
F1 score for training set: 0.855721393035
Predicting labels using AdaBoostClassifier...
Done!
Prediction time (secs): 0.019
F1 score for test set: 0.711111111111


|  Adaboost: Training set size| 100 | 200| 300|
| ----- |
| Training time (secs)|0.026 |0.022|0.026|
|Prediction time (secs)| 0.018|0.017|0.016|
|F1 score for training set|1.0 |0.902|0.866|
|F1 score for test set| 0.725|0.746|0.721|

## 5. Choosing the Best Model

- Based on the experiments you performed earlier, in 1-2 paragraphs explain to the board of supervisors what single model you chose as the best model. Which model is generally the most appropriate based on the available data, limited resources, cost, and performance?
- In 1-2 paragraphs explain to the board of supervisors in layman's terms how the final model chosen is supposed to work (for example if you chose a Decision Tree or Support Vector Machine, how does it make a prediction).
- Fine-tune the model. Use Gridsearch with at least one important parameter tuned and with at least 3 settings. Use the entire training set for this.
- What is the model's final F<sub>1</sub> score?

In [29]:
# TODO: Fine-tune your model and report the best F1 score

In [30]:
from sklearn.metrics import make_scorer, f1_score
from sklearn.metrics import mean_squared_error
from sklearn.grid_search import GridSearchCV

scoring_function = make_scorer(f1_score, pos_label='yes')

clf = tree.DecisionTreeClassifier()
#svc = svm.SVC()
#parameters = [{'C':[1e-10, 1e-5, 1e-4, 1e-3, 1**(-2.2), 1**(-2.1), 1e-2, 1**(-1.9), 1**(-1.8), 1e-1, 1], 
#               'kernel':['linear', 'poly', 'rbf', 'sigmoid']}]

#parameters = [{'C':[1e-10, 1e-5, 0.01, 0.1, 1], 
#               'kernel':['linear', 'poly', 'rbf', 'sigmoid']}]
parameters = [{'max_depth': np.arange(2, 10),'min_samples_split':[1,2,5,10,15]}]
#parameters = [{'max_depth': np.arange(3, 10)}]
#reg = GridSearchCV(svc, param_grid =parameters, scoring=scoring_function)
reg = GridSearchCV(clf, param_grid =parameters, scoring=scoring_function)

train_classifier(reg, X_train, y_train)

#reg.fit(X_train,y_train)

Training GridSearchCV...
Done!
Training time (secs): 0.658
------------------------------------------


In [31]:
reg.fit(X_train, y_train)

print(reg.best_params_)
print(f1_score(y_test, reg.predict(X_test), pos_label= 'yes'))

{'min_samples_split': 1, 'max_depth': 3}
0.786206896552
